In [152]:
import pandas as pd

In [153]:
assay = pd.read_csv("../data/assay.csv")
collar = pd.read_csv("../data/collar.csv")
survey = pd.read_csv("../data/survey.csv")

In [154]:
from desurvey import mincurve_desurvey

In [157]:
def desurvey(survey, collar):
    def get_coords(row):
        if np.isnan(row.length):
            icollar = collar[collar.DHID == row.DHID]
            return {"dhid":row.DHID, 
                "x":icollar.X.values[0], 
                "y":icollar.Y.values[0], 
                "z":icollar.Z.values[0]}
        else:
            x,y,z = mincurve_desurvey(row.length, row.AZIMUTH, row.DIP, row.AZIMUTH2, row.DIP2)
            return {"dhid":row.DHID,"x":x, "y":y, "z":-z}

    bhid_groups = survey.groupby("DHID")
    survey["length"] = bhid_groups["DEPTH"].diff()
    survey["AZIMUTH2"] = bhid_groups["AZIMUTH"].shift(1)
    survey["DIP2"] = bhid_groups["DIP"].shift(1)
    coords = survey.apply(get_coords, axis=1, result_type="expand")
    coords["z"] = coords.groupby("dhid")["z"].cumsum()
    return coords

In [158]:
desurvey(survey, collar)

,dhid,x,y,z
0,0.0,4.533054,0.000000e+00,99.727177
1,0.0,0.318121,1.947928e-17,94.082975
2,0.0,0.183302,1.122401e-17,91.262380
3,0.0,0.231747,1.419041e-17,88.445352
4,0.0,0.280124,1.715264e-17,85.632722
...,...,...,...,...
367,9.0,1.113951,6.820985e-17,77.476288
368,9.0,1.254041,7.678785e-17,69.133899
369,9.0,1.393775,8.534412e-17,60.813714
370,9.0,1.533118,9.387638e-17,52.518075
